In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install kaggle
!pip install h5py
!pip install h5pyViewer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 74 kB 2.9 MB/s 
     |████████████████████████████████| 74 kB 3.1 MB/s 
     |████████████████████████████████| 74 kB 3.0 MB/s 
     |████████████████████████████████| 74 kB 2.7 MB/s 
     |████████████████████████████████| 73 kB 1.8 MB/s 
     |████████████████████████████████| 73 kB 2.3 MB/s 
     |████████████████████████████████| 73 kB 2.3 MB/s 
     |████████████████████████████████| 73 kB 1.7 MB/s 
     |████████████████████████████████| 75 kB 4.2 MB/s 
     |████████████████████████████████| 68 kB 3.9 MB/s 
     |████████████████████████████████| 68 kB 6.5 MB/s 
     |█████████████████████████

In [ ]:
!mkdir -p ~/.kaggle/ && cp /content/drive/MyDrive/kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d fantineh/next-day-wildfire-spread

 99% 2.06G/2.08G [00:09<00:00, 189MB/s]
100% 2.08G/2.08G [00:09<00:00, 237MB/s]


In [ ]:
!unzip /content/next-day-wildfire-spread.zip

Archive:  /content/next-day-wildfire-spread.zip
  inflating: next_day_wildfire_spread_eval_00.tfrecord  
  inflating: next_day_wildfire_spread_eval_01.tfrecord  
  inflating: next_day_wildfire_spread_test_00.tfrecord  
  inflating: next_day_wildfire_spread_test_01.tfrecord  
  inflating: next_day_wildfire_spread_train_00.tfrecord  
  inflating: next_day_wildfire_spread_train_01.tfrecord  
  inflating: next_day_wildfire_spread_train_02.tfrecord  
  inflating: next_day_wildfire_spread_train_03.tfrecord  
  inflating: next_day_wildfire_spread_train_04.tfrecord  
  inflating: next_day_wildfire_spread_train_05.tfrecord  
  inflating: next_day_wildfire_spread_train_06.tfrecord  
  inflating: next_day_wildfire_spread_train_07.tfrecord  
  inflating: next_day_wildfire_spread_train_08.tfrecord  
  inflating: next_day_wildfire_spread_train_09.tfrecord  
  inflating: next_day_wildfire_spread_train_10.tfrecord  
  inflating: next_day_wildfire_spread_train_11.tfrecord  
  inflating: next_day_wildfi

In [ ]:
import re
from typing import Dict, List, Optional, Text, Tuple
import matplotlib.pyplot as plt
from matplotlib import colors
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox
import tensorflow_datasets as tfds
from keras.callbacks import ModelCheckpoint

In [ ]:
file_pattern = '/content/next_day_wildfire_spread_train*'

In [ ]:
INPUT_FEATURES = ['elevation', 'th', 'vs',  'tmmn', 'tmmx', 'sph',
                  'pr', 'pdsi', 'NDVI', 'population', 'erc', 'PrevFireMask']

OUTPUT_FEATURES = ['FireMask', ]

DATA_STATS = {'elevation': (0.0, 3141.0, 657.3003, 649.0147), 'pdsi': (-6.1298, 7.8760, -0.0053, 2.6823), 'NDVI': (-9821.0, 9996.0, 5157.625, 2466.6677), 'pr': (0.0, 44.5304, 1.7398051, 4.4828), 'sph': (0., 1., 0.0071658953, 0.0042835088), 'th': (0., 360.0, 190.3298, 72.5985), 'tmmn': (253.15, 298.9489, 281.08768, 8.9824), 'tmmx': (253.15, 315.0923, 295.17383, 9.8155), 'vs': (0.0, 10.0243, 3.8501, 1.4110), 'erc': (0.0, 106.2489, 37.3263, 20.8460), 'population': (0., 2534.0630, 25.5314, 154.7233), 'PrevFireMask': (-1., 1., 0., 1.), 'FireMask': (-1., 1., 0., 1.)} 


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import io
import imageio
from IPython.display import Image, display
from ipywidgets import widgets, Layout, HBox


In [ ]:
def input_output_images(input_img:tf.Tensor, output_img:tf.Tensor, sample_size:int, num_in_channels:int, num_out_channels:int):
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(combined,[sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img
  #return combined
#input_output_images(input_img, output_img, sample_size, num_in_channels, 1)


In [ ]:

def random_crop_input_and_output_images(input_img:tf.Tensor, output_img:tf.Tensor, sample_size:int, num_in_channels:int, num_out_channels:int,) -> Tuple[tf.Tensor, tf.Tensor]:
  combined = tf.concat([input_img, output_img], axis=2)
  combined = tf.image.random_crop(
      combined,
      [sample_size, sample_size, num_in_channels + num_out_channels])
  input_img = combined[:, :, 0:num_in_channels]
  output_img = combined[:, :, -num_out_channels:]
  return input_img, output_img


def center_crop_input_and_output_images(input_img: tf.Tensor, output_img: tf.Tensor, sample_size: int,
) -> Tuple[tf.Tensor, tf.Tensor]:
  central_fraction = sample_size / input_img.shape[0]
  input_img = tf.image.central_crop(input_img, central_fraction)
  output_img = tf.image.central_crop(output_img, central_fraction)
  return input_img, output_img

In [ ]:

def _get_base_key(key: Text) -> Text:
  match = re.match(r'[a-zA-Z]+', key)
  if match:
    return match.group(1)
  raise ValueError(
      f'The provided key does not match the expected pattern: {key}')


def _clip_and_rescale(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, _, _ = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  return tf.math.divide_no_nan((inputs - min_val), (max_val - min_val))


def _clip_and_normalize(inputs: tf.Tensor, key: Text) -> tf.Tensor:
  base_key = _get_base_key(key)
  if base_key not in DATA_STATS:
    raise ValueError(
        'No data statistics available for the requested key: {}.'.format(key))
  min_val, max_val, mean, std = DATA_STATS[base_key]
  inputs = tf.clip_by_value(inputs, min_val, max_val)
  inputs = inputs - mean
  return tf.math.divide_no_nan(inputs, std)

def _get_features_dict(
    sample_size: int,
    features: List[Text],
) -> Dict[Text, tf.io.FixedLenFeature]:
  sample_shape = [sample_size, sample_size]
  features = set(features)
  columns = [
      tf.io.FixedLenFeature(shape=sample_shape, dtype=tf.float32)
      for _ in features
  ]
  return dict(zip(features, columns))


def _parse_fn(example_proto: tf.train.Example, data_size: int, sample_size: int, num_in_channels: int, clip_and_normalize: bool, clip_and_rescale: bool, random_crop: bool, center_crop: bool,) -> Tuple[tf.Tensor, tf.Tensor]:
  if (random_crop and center_crop):
    raise ValueError('Cannot have both random_crop and center_crop be True')
  input_features, output_features = INPUT_FEATURES, OUTPUT_FEATURES
  feature_names = input_features + output_features
  features_dict = _get_features_dict(data_size, feature_names)
  features = tf.io.parse_single_example(example_proto, features_dict)

  if clip_and_normalize:
    inputs_list = [_clip_and_normalize(features.get(key), key) for key in input_features]
  elif clip_and_rescale:
    inputs_list = [_clip_and_rescale(features.get(key), key) for key in input_features]
  else:
    inputs_list = [features.get(key) for key in input_features]

  inputs_stacked = tf.stack(inputs_list, axis=0)
  input_img = tf.transpose(inputs_stacked, [1, 2, 0])

  outputs_list = [features.get(key) for key in output_features]
  assert outputs_list, 'outputs_list should not be empty'
  outputs_stacked = tf.stack(outputs_list, axis=0)

  outputs_stacked_shape = outputs_stacked.get_shape().as_list()
  assert len(outputs_stacked.shape) == 3, ('outputs_stacked should be rank 3''but dimensions of outputs_stacked'f' are {outputs_stacked_shape}')
  output_img = tf.transpose(outputs_stacked, [1, 2, 0])

  if random_crop:
    input_img, output_img = random_crop_input_and_output_images(input_img, output_img, sample_size, num_in_channels, 1)
  if center_crop:
    input_img, output_img = center_crop_input_and_output_images(input_img, output_img, sample_size)
  
  # insert empty dimension to make it work with ConvLSTM
  input_img = tf.expand_dims(input_img, axis=0)  
  output_img = tf.expand_dims(output_img, axis=0)

  return input_img, output_img


def get_dataset(file_pattern: Text, data_size: int, sample_size: int, batch_size: int, num_in_channels: int, compression_type: Text, clip_and_normalize: bool, clip_and_rescale: bool, random_crop: bool, center_crop: bool) -> tf.data.Dataset:
  if (clip_and_normalize and clip_and_rescale):
    raise ValueError('Cannot have both normalize and rescale.')
  dataset = tf.data.Dataset.list_files(file_pattern)
  dataset = dataset.interleave(lambda x: tf.data.TFRecordDataset(x, compression_type=compression_type), num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  dataset = dataset.map(lambda x: _parse_fn(x, data_size, sample_size, num_in_channels, clip_and_normalize, clip_and_rescale, random_crop, center_crop),num_parallel_calls=tf.data.experimental.AUTOTUNE)
  dataset = dataset.batch(batch_size)
  dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return dataset

In [ ]:
dataset = get_dataset(
      file_pattern,
      data_size=64,
      sample_size=32,
      batch_size=100,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

In [ ]:
val_dataset = get_dataset(
      '/content/next_day_wildfire_spread_eval*',
      data_size=64,
      sample_size=32,
      batch_size=100,
      num_in_channels=12,
      compression_type=None,
      clip_and_normalize=False,
      clip_and_rescale=False,
      random_crop=True,
      center_crop=False)

In [ ]:
# Dataset Basic Info
print(f"Size of training dataset: {len(list(dataset)) * 100}")
print(dataset)
print(f"\nSize of validation datset: {len(list(val_dataset)) * 100}")
print(val_dataset)

Size of training dataset: 15000
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 1, 32, 32, 12), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 32, 32, 1), dtype=tf.float32, name=None))>

Size of validation datset: 1900
<PrefetchDataset element_spec=(TensorSpec(shape=(None, 1, 32, 32, 12), dtype=tf.float32, name=None), TensorSpec(shape=(None, 1, 32, 32, 1), dtype=tf.float32, name=None))>


In [ ]:
# U-NET model


In [ ]:
# Construct the input layer with no definite frame size.
inp = layers.Input(shape=(1, 32, 32, 12))

# We will construct 3 `ConvLSTM2D` layers with batch normalization,
# followed by a `Conv3D` layer for the spatiotemporal outputs.
x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(5, 5),
    padding="same",
    return_sequences=True,
    activation="relu",
)(inp)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(3, 3),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.BatchNormalization()(x)
x = layers.ConvLSTM2D(
    filters=32,
    kernel_size=(1, 1),
    padding="same",
    return_sequences=True,
    activation="relu",
)(x)
x = layers.Conv3D(
    filters=1, kernel_size=(3, 3, 3), activation="sigmoid", padding="same"
)(x)

# Next, we will build the complete model and compile it.
model = keras.models.Model(inp, x)
model.compile(
    loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(),
    metrics=[tf.keras.metrics.AUC(curve='PR')]
)


In [ ]:
# # Construct the input layer with no definite frame size.
# inp = layers.Input(shape=(32, 32, 12))

# # We will construct 3 `ConvLSTM2D` layers with batch normalization,
# # followed by a `Conv3D` layer for the spatiotemporal outputs.
# x = layers.Conv2D(
#     filters=32, 
#     kernel_size=(3,3),
#     padding="same",
#     activation="relu",
# )(inp)
# x = layers.BatchNormalization()(x)
# x = layers.MaxPooling2D(
#     pool_size=(2,2),
# )(inp)
# x = layers.Conv2D(
#     filters=32,
#     kernel_size=(3, 3),
#     padding="same",
#     activation="relu",
# )(inp)
# x = layers.BatchNormalization()(x)
# x = layers.Conv2D(
#     filters=1, kernel_size=(3, 3), activation="sigmoid", padding="same"
# )(x)

# # Next, we will build the complete model and compile it.
# model = keras.models.Model(inp, x)
# model.compile(
#     loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam(learning_rate=0.001,),
#     metrics=[tf.keras.metrics.AUC(curve='PR')]
# )

In [ ]:
from timeit import default_timer as timer

class TimingCallback(keras.callbacks.Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

tcb = TimingCallback()

In [ ]:
# checkpoint = ModelCheckpoint("model.hdf5", monitor = 'val_auc', verbose = 1, save_best_only = False, period = 1)
early_stopping = keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)
reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor="val_loss", patience=5)

# Define modifiable training hyperparameters.
epochs = 5
batch_size = 100

total_time_to_train = timer()
# Fit the model to the training data.
history = model.fit(
    dataset,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_dataset,
    callbacks= [early_stopping, reduce_lr, tcb] # , checkpoint]
)
total_time_to_train = timer()

Epoch 1/5
150/150 [==============================] - 8s 49ms/step - loss: 0.2551 - auc_3: 0.0474 - val_loss: 0.0087 - val_auc_3: 0.0529
Epoch 2/5
150/150 [==============================] - 6s 42ms/step - loss: -0.0314 - auc_3: 0.0501 - val_loss: -0.0569 - val_auc_3: 0.0556
Epoch 3/5
150/150 [==============================] - 6s 43ms/step - loss: -0.1233 - auc_3: 0.0521 - val_loss: -0.0822 - val_auc_3: 0.0563
Epoch 4/5
150/150 [==============================] - 6s 43ms/step - loss: -0.2415 - auc_3: 0.0532 - val_loss: -0.2932 - val_auc_3: 0.0541
Epoch 5/5
150/150 [==============================] - 7s 43ms/step - loss: -0.3977 - auc_3: 0.0532 - val_loss: 0.1218 - val_auc_3: 0.0532


In [ ]:
print(f"Average time per epoch: {sum(tcb.logs)/len(tcb.logs)} seconds")
print(f"Total Time taken: {sum(tcb.logs)} seconds")
print(f"Total time to train: {total_time_to_train} seconds")

Average time per epoch: 23.479602731799986 seconds
Total Time taken: 117.39801365899993 seconds


In [ ]:
# history.history

In [ ]:
model.save("model")

In [ ]:
# for i in range(149):
#   features, firemask = next(iter(dataset))
#   features_np = features.numpy()
#   firemask_np = firemask.numpy()
#   features_np = np.expand_dims(features_np, axis = 1)
#   features_np.shape
#   firemask_np = np.expand_dims(firemask_np, axis = 1)
#   firemask_np.shape  
#   algo_train()
  

In [ ]:
#with h5py.File('model.hdf5', 'r') as f:
    #data = f['']
    #data = f['val_acc']
    # get the minimum value
    #ls = list(f.keys())
    #print(ls)
    #data = f.get('val_loss')
    #dataset1 = np.array(data)
    #print(dataset1)

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1, 32, 32, 12)]   0         
                                                                 
 conv_lstm2d (ConvLSTM2D)    (None, 1, 32, 32, 32)     140928    
                                                                 
 batch_normalization (BatchN  (None, 1, 32, 32, 32)    128       
 ormalization)                                                   
                                                                 
 conv_lstm2d_1 (ConvLSTM2D)  (None, 1, 32, 32, 32)     73856     
                                                                 
 batch_normalization_1 (Batc  (None, 1, 32, 32, 32)    128       
 hNormalization)                                                 
                                                                 
 conv_lstm2d_2 (ConvLSTM2D)  (None, 1, 32, 32, 32)     8320  

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.vis_utils import plot_model
#model = Sequential()
#model.add(Dense(2, input_dim=1, activation='relu'))
#model.add(Dense(1, activation='sigmoid'))
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, rankdir = 'TB', expand_nested = True)


# Viz

# New Section

In [ ]:
CMAP = colors.ListedColormap(['black', 'silver', 'orangered'])
BOUNDS = [-1, -0.1, 0.001, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)
SAMPLE = 1  # row
FIREMASK_IDX = 11  # the index of the feature with the firemask at t

In [ ]:
x_pred = np.zeros(shape=(32,32))
for i in range(32):
  for j in range(32):
    x_pred[i,j] = features_val_np[SAMPLE][0][i,j][FIREMASK_IDX]
# plt.imshow(recolor(x_pred))
x_pred = features_val_np[SAMPLE, 0, :, :, FIREMASK_IDX]
plt.imshow(x_pred, cmap=CMAP, norm=NORM)
# print(recolor(x_pred).ravel()[0:1024].reshape((32,32)))
plt.title("FIRE MASK T")
plt.show()

In [ ]:
y_true = firemask_val_np[SAMPLE].reshape([32,32])
# plt.imshow(recolor(y_true))
plt.imshow(y_true, cmap=CMAP, norm=NORM)
plt.title("FIRE MASK T+1 Ground Truth")
plt.show()

In [ ]:
x = np.expand_dims(features_val_np[SAMPLE],axis=0)
y_pred = model.predict(x)
y_pred_reshaped = y_pred.ravel().reshape((32,32))

CMAP = colors.ListedColormap(['silver', 'orangered'])
BOUNDS = [0.0, 0.25, 1]
NORM = colors.BoundaryNorm(BOUNDS, CMAP.N)

plt.imshow(y_pred_reshaped,cmap=CMAP, norm=NORM)
# plt.imshow(y_pred_reshaped)
plt.colorbar()
plt.title("FIRE MASK T+1 Predicted")
plt.show()

In [ ]:
fig, axs = plt.subplots(1,3)
fig.suptitle('Firemasks')
axs[0].imshow(x_pred)
axs[0].set_title("Input")
axs[1].imshow(y_true)
axs[1].set_title("Ground Truth")
axs[2].imshow(y_pred_reshaped)
axs[2].set_title("Predicted")
fig.colorbar()
fig.axis("off")